<a href="https://colab.research.google.com/github/volodymyr-podkopaiev/portfolio/blob/main/Portfolio_Project_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
from google.cloud import bigquery
project_id = 'data-analytics-mate'
client = bigquery.Client(project=project_id)

In [ ]:
sql_query = """
WITH session_info AS(
 SELECT
   s.date AS date,
   s.ga_session_id AS ga_session_id,
   sp.country AS country,
   sp.device AS device,
   sp.continent AS continent,
   sp.channel AS channel,
   t.test AS test,
   t.test_group AS test_group
 FROM `DA.ab_test` AS t
 JOIN `DA.session` AS s ON s.ga_session_id = t.ga_session_id
 JOIN `DA.session_params` AS sp ON sp.ga_session_id = t.ga_session_id
),
session_with_orders AS(
SELECT
 s_info.date AS date,
 s_info.country AS country,
 s_info.device AS device,
 s_info.continent AS continent,
 s_info.channel AS channel,
 s_info.test AS test,
 s_info.test_group AS test_group,
 count(DISTINCT o.ga_session_id) AS session_with_orders
FROM `DA.order` AS o
JOIN session_info AS s_info ON o.ga_session_id = s_info.ga_session_id
GROUP BY
 s_info.date,
 s_info.country,
 s_info.device,
 s_info.continent,
 s_info.channel,
 s_info.test,
 s_info.test_group
),
events AS(
SELECT
 s_info.date AS date,
 s_info.country AS country,
 s_info.device AS device,
 s_info.continent AS continent,
 s_info.channel AS channel,
 s_info.test AS test,
 s_info.test_group AS test_group,
 e.event_name,
 count(e.ga_session_id) AS event_cnt
FROM `DA.event_params` AS e
JOIN session_info AS s_info ON s_info.ga_session_id = e.ga_session_id
GROUP BY
 s_info.date,
 s_info.country,
 s_info.device,
 s_info.continent,
 s_info.channel,
 s_info.test,
 s_info.test_group,
 e.event_name
),
session AS(
SELECT
 s_info.date AS date,
 s_info.country AS country,
 s_info.device AS device,
 s_info.continent AS continent,
 s_info.channel AS channel,
 s_info.test AS test,
 s_info.test_group AS test_group,
 COUNT(DISTINCT s_info.ga_session_id) AS session_cnt
FROM session_info AS s_info
GROUP BY
 s_info.date,
 s_info.country,
 s_info.device,
 s_info.continent,
 s_info.channel,
 s_info.test,
 s_info.test_group
),
account AS(
SELECT
 s_info.date,
 s_info.country,
 s_info.device,
 s_info.continent,
 s_info.channel,
 s_info.test,
 s_info.test_group,
 COUNT(DISTINCT accs.ga_session_id) AS new_account_cnt
FROM `DA.account_session` AS accs
JOIN session_info AS s_info ON s_info.ga_session_id = accs.ga_session_id
GROUP BY
 s_info.date,
 s_info.country,
 s_info.device,
 s_info.continent,
 s_info.channel,
 s_info.test,
 s_info.test_group
)


SELECT
 swo.date AS date,
 swo.country AS country,
 swo.device AS device,
 swo.continent AS continent,
 swo.channel AS channel,
 swo.test AS test,
 swo.test_group AS test_group,
 'session_with_orders' AS event_name,
 swo.session_with_orders AS value
FROM session_with_orders AS swo
UNION ALL
SELECT
 ev.date AS date,
 ev.country AS country,
 ev.device AS device,
 ev.continent AS continent,
 ev.channel AS channel,
 ev.test AS test,
 ev.test_group AS test_group,
 event_name,
 ev.event_cnt AS value
FROM events AS ev
UNION ALL
SELECT
 sess.date AS date,
 sess.country AS country,
 sess.device AS device,
 sess.continent AS continent,
 sess.channel AS channel,
 sess.test AS test,
 sess.test_group AS test_group,
 'session' AS event_name,
 sess.session_cnt AS value
FROM session AS sess
UNION ALL
SELECT
 acc.date AS date,
 acc.country AS country,
 acc.device AS device,
 acc.continent AS continent,
 acc.channel AS channel,
 acc.test AS test,
 acc.test_group AS test_group,
 'new_account' AS event_name,
 acc.new_account_cnt AS value
FROM account AS acc
"""
df = client.query(sql_query).to_dataframe()
display(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800996 entries, 0 to 800995
Data columns (total 9 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   date        800996 non-null  dbdate
 1   country     800996 non-null  object
 2   device      800996 non-null  object
 3   continent   800996 non-null  object
 4   channel     800996 non-null  object
 5   test        800996 non-null  Int64 
 6   test_group  800996 non-null  Int64 
 7   event_name  800996 non-null  object
 8   value       800996 non-null  Int64 
dtypes: Int64(3), dbdate(1), object(5)
memory usage: 57.3+ MB


None

In [ ]:
import pandas as pd
df['date'] = pd.to_datetime(df['date'])
for column in df.columns:
  if df[column].dtype == 'object':
    df[column] = df[column].astype('category')
display(df.info())
from google.colab import drive
drive.mount('/content/drive')
df.to_csv('/content/drive/MyDrive/MateAcademy/Projects/raw_data.csv', index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800996 entries, 0 to 800995
Data columns (total 9 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   date        800996 non-null  datetime64[ns]
 1   country     800996 non-null  category      
 2   device      800996 non-null  category      
 3   continent   800996 non-null  category      
 4   channel     800996 non-null  category      
 5   test        800996 non-null  Int64         
 6   test_group  800996 non-null  Int64         
 7   event_name  800996 non-null  category      
 8   value       800996 non-null  Int64         
dtypes: Int64(3), category(5), datetime64[ns](1)
memory usage: 30.6 MB


None

Mounted at /content/drive


In [ ]:
# Визначаємо метрики за якими будемо розраховувати результати тесту
metrics = [
  'add_payment_info',
  'add_shipping_info',
  'begin_checkout',
  'new_account',
  'session'
]

def get_metrics(df, metrics, **kwargs):
  result = pd.DataFrame()
  for key, value in kwargs.items():
    df = df[df[key] == value]
  result = pd.pivot_table(df, index='event_name', columns='test_group', values='value', aggfunc='sum', observed=True, fill_value=0)
  result = result.reindex(metrics, fill_value=0)
  for key, value in kwargs.items():
    if key == 'test':
      result['test_number'] = value
    else:
      result[key] = value
  return result.reset_index()

In [ ]:
from itertools import product
channels = df['channel'].unique().tolist()
devices = df['device'].unique().tolist()
test_numbers = df['test'].unique().tolist()

def create_test_combinations(test_numbers, **filters):
    """
    Створюємо список з фільтрами де test_number обов'язковий,
    можливо додавання параметрів (channel, device, country та іншіх)

    Приклад:
    create_test_combinations([1, 2], channel=['web', 'app'], device=['ios', 'android'])
    """
    combinations = []

    # Додаємо None до кожного фільтру як що він не обов'язковий
    filter_keys = filters.keys()
    filter_values = [[None] + v for v in filters.values()]

    # комбінація всіх можливих фільтрів з test_numbers
    for test in test_numbers:
        for values in product(*filter_values):
            filter_dict = {'test': test}
            for key, val in zip(filter_keys, values):
                if val is not None:
                    filter_dict[key] = val
            combinations.append(filter_dict)

    return combinations

all_combinations = create_test_combinations(test_numbers, device=devices)

# Отримуємо результати всіх тестів які нас цікавлять
all_test = pd.DataFrame()
for combinations in all_combinations:
  result = get_metrics(df, metrics, **combinations)
  if result is not None and not result.empty:
    all_test = pd.concat([all_test, result], ignore_index=True)

In [ ]:
# Функція розрахунку статистичних показників по тестах
def calculate_statistics(df):
  result_test = pd.DataFrame()
  result_test = df[df['event_name'] != 'session'].copy()
  result_test.rename(columns={'event_name':'numerator_event', 1:'numerator_count_1', 2:'numerator_count_2'}, inplace=True)
  result_test['denominator_event'] = 'session'
  result_test['denominator_count_1'] = df[df['event_name'] == 'session'][1].values[0]
  result_test['denominator_count_2'] = df[df['event_name'] == 'session'][2].values[0]
  result_test['conversion_rate_1'] = result_test['numerator_count_1'] / result_test['denominator_count_1']
  result_test['conversion_rate_2'] = result_test['numerator_count_2'] / result_test['denominator_count_2']
  result_test['metric_change'] = (result_test['conversion_rate_2'] / result_test['conversion_rate_1'] - 1) * 100
  result_test = result_test.reset_index(drop=True)
  return result_test

In [ ]:
from scipy.stats import chi2_contingency
# Фукнція розрахунку статистичної значущості
def ab_test_chi2(row):
    table = [
        [row['numerator_count_1'], row['denominator_count_1'] - row['numerator_count_1']],
        [row['numerator_count_2'], row['denominator_count_2'] - row['numerator_count_2']]
    ]
    chi2, p, dof, expected = chi2_contingency(table, correction=False)
    return chi2, p

result_test = calculate_statistics(all_test)
result_test[['z_stat', 'p_value']] = pd.DataFrame(result_test.apply(ab_test_chi2, axis=1).tolist(), index=result_test.index)
result_test['significant'] = result_test['p_value'] < 0.05
display(result_test[(result_test['p_value'] < 0.05) & (result_test['metric_change'] > 10)])

test_group,numerator_event,numerator_count_1,numerator_count_2,test_number,device,denominator_event,denominator_count_1,denominator_count_2,conversion_rate_1,conversion_rate_2,metric_change,z_stat,p_value,significant
0,add_payment_info,1988,2229,1,NaN,session,45362,45193,0.043825,0.049322,12.542021,15.404715,0.000087,True
4,add_payment_info,810,942,1,mobile,session,45362,45193,0.017856,0.020844,16.731188,10.650090,0.001101,True
8,add_payment_info,1130,1256,1,desktop,session,45362,45193,0.024911,0.027792,11.566091,7.325465,0.006798,True
9,add_shipping_info,1711,1916,1,desktop,session,45362,45193,0.037719,0.042396,12.400053,12.880458,0.000332,True
10,begin_checkout,2108,2404,1,desktop,session,45362,45193,0.046471,0.053194,14.468207,21.616103,0.000003,True


З чотирьох тестів зріст більш ніж на 10% та стратистичну значущість маємо лише в першому тесті за метрикою add_payment_info. Також можно виділити зріст конверсії у першому тесті з трафіку отриманого з десктопів.

In [ ]:
# Експортуємо датафрейм у текcтовий файл для роботи з Tableu
from google.colab import drive
drive.mount('/content/drive')
result_test.to_csv('/content/drive/MyDrive/MateAcademy/Projects/portfolio_project2.csv', index=False)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Посилання на датасет

https://drive.google.com/file/d/1QaZ5XeUmiv6RIoPXxZjnIpyi74PVhZAa/view?usp=sharing

Посилання на дашборд Tableau
https://public.tableau.com/views/CreateYourABTestingTool_17545095017710/ABtest?:language=en-US&publish=yes&:sid=&:redirect=auth&:display_count=n&:origin=viz_share_link